In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
import matplotlib
import matplotlib.pyplot as plt

In [2]:
plt.style.use('ggplot')
matplotlib.rcParams['figure.dpi'] = 100

## Feature Engineering

In [3]:
path = ''
wh = pd.read_csv(path+'wh_cleaned.csv', memory_map=True)
perf = pd.read_csv(path+'perf_cleaned.csv', memory_map=True)
prod = pd.read_csv(path+'prod_cleaned2.csv', memory_map=True)

C:\Users\Nguyen\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Assign correct dtypes
for var in wh.columns[wh.columns.str.contains('Date')].values: 
    wh[var] = pd.to_datetime(wh[var])
wh['CurrentOperatorParent'] = wh['CurrentOperatorParent'].astype('object')
wh['LicenseeParentCompany'] = wh['LicenseeParentCompany'].astype('object')

perf['ActivityDate'] = pd.to_datetime(perf['ActivityDate'])
perf['ObservationNumber'] = perf['ObservationNumber'].astype('int64')
perf['PerfShots'] = perf['PerfShots'].astype('int64')

prod['ProdPeriod'] = pd.to_datetime(prod['ProdPeriod'])

In [5]:
datatype = pd.DataFrame(wh.dtypes, columns=['pdtype'])
datatype['nunique'] = wh.nunique()
datatype['nnan'] = wh.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8694,0
Province,object,2,0
LicenceNumber,object,8687,0
UWI,object,8694,0
CurrentOperator,object,106,0
CurrentOperatorParent,object,104,0
CurrentOperatorParentName,object,104,0
Licensee,object,103,0
LicenseeParentCompany,object,103,0
LicenseeParentCompanyName,object,103,0


In [6]:
datatype = pd.DataFrame(perf.dtypes, columns=['pdtype'])
datatype['nunique'] = perf.nunique()
datatype['nnan'] = perf.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8415,0
EPAPTId,int64,376653,0
ObservationNumber,int64,686,0
ActivityDate,datetime64[ns],1830,0
ActivityType,object,18,0
IntervalTop,float64,52870,0
IntervalBase,float64,54848,0
PerfShots,int64,29,0


In [7]:
datatype = pd.DataFrame(prod.dtypes, columns=['pdtype'])
datatype['nunique'] = prod.nunique()
datatype['nnan'] = prod.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8661,0
ProdPeriod,datetime64[ns],61,0
Condensate,float64,10212,0
Duration,float64,744,0
Gas,float64,27296,0
Oil,float64,11585,0
Water,float64,13556,0


### perf

In [8]:
perf_agg = pd.DataFrame()

In [9]:
comlist = np.setdiff1d(perf.ActivityType.unique(), ['Bridge Plug - No Cement', 'Cement Plug', 'Cement Squeeze', 
                                                    'Packing Device Capped w/Cement', 'Remedial Casing Cementing'])
perf_agg['CompletionTop_perf'] = perf[perf.ActivityType.isin(comlist)].groupby('EPAssetsId')['IntervalTop'].min()
perf_agg['CompletionEnd_perf'] = perf[perf.ActivityType.isin(comlist)].groupby('EPAssetsId')['IntervalBase'].max()

perf_agg['CompletionStartDate_perf'] = perf.groupby('EPAssetsId')['ActivityDate'].min()
perf_agg['CompletionEndDate_perf'] = perf.groupby('EPAssetsId')['ActivityDate'].max()
perf_agg['CompletionTime_perf'] = (perf_agg['CompletionEndDate_perf'] - perf_agg['CompletionStartDate_perf']).dt.days

perf_agg['nCompletion_perf'] = perf.groupby('EPAssetsId')['ObservationNumber'].nunique()
perf_agg['nCompletionType_perf'] = perf.groupby('EPAssetsId')['ActivityType'].nunique()

In [10]:
perf_agg.reset_index(inplace=True)
perf_agg.head()

,EPAssetsId,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf
0,1145472,835.0,1510.5,2015-02-22,2015-02-22,0,1,1
1,1145481,913.0,1480.5,2015-01-22,2015-01-22,0,19,1
2,1145482,896.0,1492.5,2015-02-15,2015-02-15,0,1,1
3,1145487,870.0,1447.0,2015-02-22,2015-02-22,0,1,1
4,1145496,925.0,1490.5,2015-01-21,2015-01-21,0,19,1


### prod

In [11]:
prod_agg = pd.DataFrame()

In [12]:
for var in ['Condensate', 'Gas', 'Oil', 'Water']: 
    prod[var+'Rate_prod'] = prod[var]/prod['Duration']*24

In [13]:
prod_agg['StartDate_prod'] = prod.groupby('EPAssetsId')['ProdPeriod'].min() + pd.DateOffset(days=1, months=-1)
prod_agg['CumDuration_prod'] = prod.groupby('EPAssetsId')['Duration'].sum()
prod_agg['CumProdPeriod_prod'] = prod.groupby('EPAssetsId')['ProdPeriod'].apply(lambda x: (max(x)-min(x)))
prod_agg['CumProdPeriod_prod'] = prod_agg['CumProdPeriod_prod'].dt.days

for var in ['Condensate', 'Gas', 'Oil', 'Water']: 
    prod_agg['Cum'+var+'_prod'] = prod.groupby('EPAssetsId')[var].sum()
    prod_agg['Avg'+var+'_prod'] = prod_agg['Cum'+var+'_prod']/prod_agg['CumDuration_prod']*24
    prod_agg['Max'+var+'_prod'] = prod.groupby('EPAssetsId')[var+'Rate_prod'].max()
    prod_agg['Min'+var+'_prod'] = prod.groupby('EPAssetsId')[var+'Rate_prod'].min()

In [14]:
prod_agg.reset_index(inplace=True)
prod_agg.head()

,EPAssetsId,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod
0,1145472,2015-03-01,42942.0,1767,0.0,0.0,0.0,0.0,8.585028,0.004798,0.019936,0.000716,15956.59362,8.918035,42.790930,0.713977,3349.31850,1.871912,13.249159,0.000000
1,1145481,2015-01-29,40768.0,1767,0.0,0.0,0.0,0.0,7.108869,0.004185,0.027289,0.000228,8735.27424,5.142430,42.141660,0.040579,4367.63712,2.571215,33.793380,0.202897
2,1145482,2015-03-01,42817.0,1767,0.0,0.0,0.0,0.0,7.737473,0.004337,0.018952,0.000631,8817.67062,4.942525,23.314192,1.379698,2815.31448,1.578054,42.770640,0.304345
3,1145487,2015-01-29,42855.0,1798,0.0,0.0,0.0,0.0,6.300160,0.003528,0.018875,0.000342,14170.91940,7.936112,35.374703,1.299892,2829.78102,1.584757,184.920120,0.466663
4,1145496,2015-01-29,43114.0,1798,0.0,0.0,0.0,0.0,15.948164,0.008878,0.018329,0.005000,7876.71654,4.384682,18.450080,0.464898,1522.76058,0.847666,6.084851,0.121738


### Joining

In [17]:
main = pd.concat([df.set_index('EPAssetsId') for df in [wh, perf_agg, prod_agg]], axis=1, join='outer').reset_index()
main.head()

,EPAssetsId,Province,LicenceNumber,UWI,CurrentOperator,CurrentOperatorParent,CurrentOperatorParentName,Licensee,LicenseeParentCompany,LicenseeParentCompanyName,LicenceDate,CurrentStatus,CurrentStatusStandardised,WellType,WellTypeStandardised,ConfidentialReleaseDate,WellName,Formation,Field,Pool,Surf_Location,Surf_Township,Surf_Meridian,Surf_Range,Surf_Section,Surf_LSD,Surf_Longitude,Surf_Latitude,Surf_TownshipRange,Surf_QuarterSection,BH_Location,BH_TownshipRange,BH_QuarterSection,BH_Longitude,BH_Latitude,BH_Township,BH_Meridian,BH_Range,BH_Section,BH_LSD,GroundElevation,KBElevation,TotalDepth,LaheeClass,Confidential,SurfaceOwner,DrillingContractor,SpudDate,FinalDrillDate,RigReleaseDate,DaysDrilling,DrillMetresPerDay,TVD,WellProfile,RegulatoryAgency,PSACAreaCode,PSACAreaName,ProjectedDepth,StatusDate,StatusSource,UnitID,UnitName,UnitFlag,Municipality,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod
0,1145472,Saskatchewan,14A342,193120603021W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2014-01-24,Active,Active,Oil Production,Oil,2015-03-16,CPEC AVON HILL HZ 11A10-6-5C12-6-30-21,Viking,Avon Hill,Avon Hill Viking,10-06-030-21W3,30,W3,21,6,10,-108.942475,51.542038,030-21-W3,NE,12-06-030-21-W3,030-21-W3,NW,-108.955296,51.542254,30,W3,21,6,12,691.2,695.7,1518.0,Development,Non-Confidential,Unspecified,Precision Drilling Corporation,2015-01-16,2015-02-14,2015-02-15,29,52.344828,NaN,Horizontal,SKER,SK2,Southwestern Saskatchewan,705.0,2014-01-24,SKER,Unspecified,Unspecified,No,Kindersley,835.0,1510.5,2015-02-22,2015-02-22,0.0,1.0,1.0,2015-03-01,42942.0,1767.0,0.0,0.0,0.0,0.0,8.585028,0.004798,0.019936,0.000716,15956.59362,8.918035,42.790930,0.713977,3349.31850,1.871912,13.249159,0.000000
1,1145481,Saskatchewan,14B018,193083202922W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2014-03-02,Active,Active,Oil Production,Oil,2016-01-18,CPEC AVON HILL HZ 11A6-32-2D8-32-29-22,Viking,Avon Hill,Avon Hill Viking,06-32-029-22W3,29,W3,22,32,6,-109.065286,51.523909,029-22-W3,SW,08-32-029-22-W3,029-22-W3,SE,-109.052504,51.524055,29,W3,22,32,8,712.2,716.7,1504.0,Development,Non-Confidential,Unspecified,Precision Drilling Corporation,2015-01-16,2015-01-18,2015-01-19,2,752.000000,738.3,Horizontal,SKER,SK2,Southwestern Saskatchewan,724.0,2014-03-02,SKER,Unspecified,Unspecified,No,Kindersley,913.0,1480.5,2015-01-22,2015-01-22,0.0,19.0,1.0,2015-01-29,40768.0,1767.0,0.0,0.0,0.0,0.0,7.108869,0.004185,0.027289,0.000228,8735.27424,5.142430,42.141660,0.040579,4367.63712,2.571215,33.793380,0.202897
2,1145482,Saskatchewan,14B020,193093402922W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2014-03-02,Active,Active,Oil Production,Oil,2016-01-30,CPEC AVON HILL HZ 3A11-34-6D9-34-29-22,Viking,Avon Hill,Avon Hill Viking,11-34-029-22W3,29,W3,22,34,11,-109.018393,51.527626,029-22-W3,NW,09-34-029-22-W3,029-22-W3,NE,-109.005705,51.527585,29,W3,22,34,9,703.4,707.9,1503.0,Development,Non-Confidential,Unspecified,Precision Drilling Corporation,2015-10-01,2015-01-30,2015-01-31,20,75.150000,735.6,Horizontal,SKER,SK2,Southwestern Saskatchewan,737.0,2014-03-02,SKER,Unspecified,Unspecified,No,Kindersley,896.0,1492.5,2015-02-15,2015-02-15,0.0,1.0,1.0,2015-03-01,42817.0,1767.0,0.0,0.0,0.0,0.0,7.737473,0.004337,0.018952,0.000631,8817.67062,4.942525,23.314192,1.379698,2815.31448,1.578054,42.770640,0.304345
3,1145487,Saskatchewan,14B015,193093602922W300,Crescent Point Re

In [18]:
# Sanity check if any vertical wells are mislabelled
print(main[(main.TotalDepth==main.TVD)&(main.WellProfile!='Vertical')].EPAssetsId)
print(main[(main.Surf_Latitude==main.BH_Latitude)&(main.Surf_Latitude==main.BH_Latitude)&(main.WellProfile!='Vertical')].EPAssetsId)

4348    2585178
Name: EPAssetsId, dtype: int64
4348    2585178
Name: EPAssetsId, dtype: int64


In [19]:
# Fix this well profile
main.loc[main.EPAssetsId==2585178, 'WellProfile'] = 'Vertical'

In [20]:
# Drop wells with questionable TVD
main = main[main.EPAssetsId.isin([1151263, 1152677, 1151511, 1151468, 1173448])==False].reset_index(drop=True)

In [21]:
# nnan for _perf and _prod are expected to 279 and 33
datatype = pd.DataFrame(main.dtypes, columns=['pdtype'])
datatype['nunique'] = main.nunique()
datatype['nnan'] = main.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,8689,0
Province,object,2,0
LicenceNumber,object,8682,0
UWI,object,8689,0
CurrentOperator,object,106,0
CurrentOperatorParent,object,104,0
CurrentOperatorParentName,object,104,0
Licensee,object,103,0
LicenseeParentCompany,object,103,0
LicenseeParentCompanyName,object,103,0


### Feature Generation

#### Functions

In [24]:
def EarthRadius(lat1, lat2, Re=6378.14e3, Rp=6356.75e3): 
    '''Calculate Earth radius (m) as a function of latitudes'''
    lat = (lat1 + lat2)/2
    lat = np.deg2rad(lat)
    R = np.sqrt(((Re**2*np.cos(lat))**2 + (Rp**2*np.sin(lat))**2)/((Re*np.cos(lat))**2 + (Rp*np.sin(lat))**2))
    return R

In [25]:
def HaversineDistance(lat1, lon1, lat2, lon2): 
    '''Calculate great-circle distance (m)'''
    lat1, lon1 = np.deg2rad(lat1), np.deg2rad(lon1)
    lat2, lon2 = np.deg2rad(lat2), np.deg2rad(lon2)
    hvs = np.sin((lat2-lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2-lon1)/2)**2
    theta = np.arcsin(np.sqrt(hvs))
#     R = 6371e3
    R = EarthRadius(lat1, lat2)
    return 2*R*theta

In [26]:
def Azimuth(lat1, lon1, lat2, lon2): 
    '''Calculate azimuth (degree from north)
    Start point should be surface lat and lon'''
    lat1, lon1 = np.deg2rad(lat1), np.deg2rad(lon1)
    lat2, lon2 = np.deg2rad(lat2), np.deg2rad(lon2)
    x = np.sin(lon2 - lon1)*np.cos(lat2)
    y = np.cos(lat1)*np.sin(lat2) - (np.sin(lat1)*np.cos(lat2)*np.cos(lon2-lon1))
    azimuth = np.arctan2(x, y)
    return np.rad2deg(azimuth)

In [29]:
main_add = main.copy(deep=True)

#### Numerical

##### Distance Matrices

In [30]:
# Geo distance matrix
# Haversine takes too long, use Euclidean instead
from scipy.spatial.distance import cdist
df = main_add[['Surf_Latitude', 'Surf_Longitude', 'BH_Latitude', 'BH_Longitude']].copy(deep=True).values
distance_matrix = cdist(df, df, metric='euclidean')
print(np.mean(distance_matrix))
print(np.min(distance_matrix))
print(np.max(distance_matrix))

6.787474504859801
0.0
18.567052397196566


In [31]:
# Well profile distance matrix
df = main_add['WellProfile'].factorize()[0].reshape(-1,1)
wellprofile_matrix = cdist(df, df, metric='euclidean')
print(np.mean(wellprofile_matrix))
print(np.min(wellprofile_matrix))
print(np.max(wellprofile_matrix))

0.6602027072750014
0.0
2.0


##### FillNA perf with neighor

In [32]:
# For the purpose of fillna perf and prod, wellprofile is needed
totaldistance_matrix = distance_matrix + wellprofile_matrix*1000

In [33]:
# train+test x train+test
n_neighbors = 50
neighbors = np.empty((totaldistance_matrix.shape[0], n_neighbors))
for i in range(0, totaldistance_matrix.shape[0]): 
    neighbors[i, :] = list(main_add.EPAssetsId.values[np.argsort(totaldistance_matrix[i, :])[1:n_neighbors+1]])

In [34]:
# fillna CompletionTop_perf, CompletionEnd_perf, LateralLength_pewh (x2)
# fillna for other perf variables by the way
for i in range(2): 
    for index in main_add[main_add.CompletionTop_perf.isnull()].index: 
        mask = main_add.EPAssetsId.isin(neighbors[index, :10])
        main_add.loc[main_add.index==index, 'CompletionTop_perf'] = main_add.loc[mask, 'CompletionTop_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionEnd_perf'] = main_add.loc[mask, 'CompletionEnd_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionStartDate_perf'] = main_add.loc[mask, 'CompletionStartDate_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionEndDate_perf'] = main_add.loc[mask, 'CompletionEndDate_perf'].mean()
        main_add.loc[main_add.index==index, 'CompletionTime_perf'] = main_add.loc[mask, 'CompletionTime_perf'].mean()
        main_add.loc[main_add.index==index, 'nCompletion_perf'] = main_add.loc[mask, 'nCompletion_perf'].mean()
        main_add.loc[main_add.index==index, 'nCompletionType_perf'] = main_add.loc[mask, 'nCompletionType_perf'].mean()
    print(main_add.CompletionTop_perf.isnull().sum())
    print(main_add.CompletionEnd_perf.isnull().sum())
    print(main_add.CompletionStartDate_perf.isnull().sum())
    print(main_add.CompletionEndDate_perf.isnull().sum())
    print(main_add.CompletionTime_perf.isnull().sum())
    print(main_add.nCompletion_perf.isnull().sum())
    print(main_add.nCompletionType_perf.isnull().sum())

1
1
1
1
1
1
1
0
0
0
0
0
0
0


In [36]:
# fillna neighbor rates & volumes features
varlist = main_add.columns[main_add.columns.str.contains('_prod')]
for var in varlist:
    for index in main_add[main_add[var].isnull()].index:
        mask = main_add.EPAssetsId.isin(neighbors[index, :10])
        main_add.loc[main_add.index == index, var] = main_add.loc[mask, var].mean()
        main_add.loc[main_add.index == index, var] = main_add.loc[mask, var].min()
        main_add.loc[main_add.index == index, var] = main_add.loc[mask, var].max()
    print(main_add[var].isnull().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


##### TVDSS_calc

In [37]:
# Lateral length 
main_add['LateralLength_pewh'] = np.max([main_add.TotalDepth - main_add.CompletionTop_perf, 
                                         main_add.CompletionEnd_perf - main_add.CompletionTop_perf], axis=0)
# Azimuth
main_add['Azimuth_ltln'] = Azimuth(main_add.Surf_Latitude, main_add.Surf_Longitude,
                                   main_add.BH_Latitude, main_add.BH_Longitude)
# Horizontal Displacement
main_add['HorizontalDisplacement_hvs'] = HaversineDistance(main_add.Surf_Latitude, main_add.Surf_Longitude,
                                                           main_add.BH_Latitude, main_add.BH_Longitude)

In [38]:
# Sanity check
main_add[main_add.HorizontalDisplacement_hvs>(main_add.TotalDepth**2-main_add.TVD**2)**0.5]

,EPAssetsId,Province,LicenceNumber,UWI,CurrentOperator,CurrentOperatorParent,CurrentOperatorParentName,Licensee,LicenseeParentCompany,LicenseeParentCompanyName,LicenceDate,CurrentStatus,CurrentStatusStandardised,WellType,WellTypeStandardised,ConfidentialReleaseDate,WellName,Formation,Field,Pool,Surf_Location,Surf_Township,Surf_Meridian,Surf_Range,Surf_Section,Surf_LSD,Surf_Longitude,Surf_Latitude,Surf_TownshipRange,Surf_QuarterSection,BH_Location,BH_TownshipRange,BH_QuarterSection,BH_Longitude,BH_Latitude,BH_Township,BH_Meridian,BH_Range,BH_Section,BH_LSD,GroundElevation,KBElevation,TotalDepth,LaheeClass,Confidential,SurfaceOwner,DrillingContractor,SpudDate,FinalDrillDate,RigReleaseDate,DaysDrilling,DrillMetresPerDay,TVD,WellProfile,RegulatoryAgency,PSACAreaCode,PSACAreaName,ProjectedDepth,StatusDate,StatusSource,UnitID,UnitName,UnitFlag,Municipality,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod,LateralLength_pewh,Azimuth_ltln,HorizontalDisplacement_hvs
1732,1163751,Saskatchewan,68067,102161503223W300,Teine Energy Ltd.,1576,Teine Energy Ltd.,Teine Energy Ltd.,1576,Teine Energy Ltd.,2017-01-20,Active,Active,Oil Production,Oil,2017-03-18,TEINE HZ 16-15-32-23,Viking,Kerrobert,Kerrobert Viking,08-15-032-23W3,32,W3,23,15,8,-109.1755,51.741222,032-23-W3,SE,16-15-032-23-W3,032-23-W3,NE,-109.17484,51.749925,32,W3,23,15,16,702.3,706.6,888.0,Development,Non-Confidential,Unspecified,Savanna Drilling Corp.,2017-06-02,2017-02-15,2017-02-15,2,444.0,698.11,Horizontal,SKER,SK2,Southwestern Saskatchewan,721.0,2017-01-20,SKER,Unspecified,Unspecified,No,Oakdale,959.0,1495.5,2017-02-28,2017-02-28,0.0,1.0,1.0,2017-03-01,25338.0,1036.0,0.0,0.0,0.0,0.0,30.6144,0.028998,0.082477,0.003178,22717.49964,21.517878,57.174282,4.892454,1732.21092,1.64074,3.855039,0.0,536.5,2.688109,969.8561


In [39]:
# Fix the well's TotalDepth as this might be a typo (take CompletionEnd_perf as TD)
mask = (main_add.HorizontalDisplacement_hvs>(main_add.TotalDepth**2-main_add.TVD**2)**0.5) & (main_add.WellProfile=='Horizontal')
main_add.loc[mask, 'TotalDepth'] = main_add.loc[mask, 'CompletionEnd_perf']

In [40]:
# Another sanity check
main_add[main_add.HorizontalDisplacement_hvs<=main_add.TotalDepth-main_add.TVD*np.pi/2]

,EPAssetsId,Province,LicenceNumber,UWI,CurrentOperator,CurrentOperatorParent,CurrentOperatorParentName,Licensee,LicenseeParentCompany,LicenseeParentCompanyName,LicenceDate,CurrentStatus,CurrentStatusStandardised,WellType,WellTypeStandardised,ConfidentialReleaseDate,WellName,Formation,Field,Pool,Surf_Location,Surf_Township,Surf_Meridian,Surf_Range,Surf_Section,Surf_LSD,Surf_Longitude,Surf_Latitude,Surf_TownshipRange,Surf_QuarterSection,BH_Location,BH_TownshipRange,BH_QuarterSection,BH_Longitude,BH_Latitude,BH_Township,BH_Meridian,BH_Range,BH_Section,BH_LSD,GroundElevation,KBElevation,TotalDepth,LaheeClass,Confidential,SurfaceOwner,DrillingContractor,SpudDate,FinalDrillDate,RigReleaseDate,DaysDrilling,DrillMetresPerDay,TVD,WellProfile,RegulatoryAgency,PSACAreaCode,PSACAreaName,ProjectedDepth,StatusDate,StatusSource,UnitID,UnitName,UnitFlag,Municipality,CompletionTop_perf,CompletionEnd_perf,CompletionStartDate_perf,CompletionEndDate_perf,CompletionTime_perf,nCompletion_perf,nCompletionType_perf,StartDate_prod,CumDuration_prod,CumProdPeriod_prod,CumCondensate_prod,AvgCondensate_prod,MaxCondensate_prod,MinCondensate_prod,CumGas_prod,AvgGas_prod,MaxGas_prod,MinGas_prod,CumOil_prod,AvgOil_prod,MaxOil_prod,MinOil_prod,CumWater_prod,AvgWater_prod,MaxWater_prod,MinWater_prod,LateralLength_pewh,Azimuth_ltln,HorizontalDisplacement_hvs
3309,1170677,Saskatchewan,97847,102070103119W300,Crescent Point Resources Partnership,168,Crescent Point Energy Corp,Crescent Point Energy Corp.,168,Crescent Point Energy Corp,2018-10-07,Active,Active,Oil Production,Oil,2018-12-28,CPEC HZ 7-1-31-19,Viking,Dodsland,Dodsland Viking,08-02-031-19W3,31,W3,19,2,8,-108.58093,51.626115,031-19-W3,SE,07-01-031-19-W3,031-19-W3,SE,-108.56257,51.624438,31,W3,19,1,7,707.6,711.8,1990.0,Development,Non-Confidential,Unspecified,Savanna Drilling Corp.,2018-11-25,2018-11-27,2018-11-28,2,995.0,134.0,Horizontal,SKER,SK2,Southwestern Saskatchewan,730.0,2018-10-07,SKER,Unspecified,Unspecified,No,Winslow,915.0,1971.5,2018-12-13,2018-12-13,0.0,1.0,1.0,2018-12-01,9792.0,396.0,0.0,0.0,0.0,0.0,11.09591,0.027196,0.082738,0.002051,15594.30114,38.221326,72.422554,7.952104,3843.69678,9.420825,73.950077,3.185479,1075.0,98.366018,1282.439375


In [41]:
# Fix the well's TVD as this might be a typo (734, instead of 134)
mask = (main_add.HorizontalDisplacement_hvs<main_add.TotalDepth-main_add.TVD*np.pi/2) & (main_add.WellProfile=='Horizontal')
main_add.loc[mask, 'TVD'] = 734

In [42]:
# TVD_calc from horizontal model
mask1 = main_add.WellProfile.isin(['Horizontal', 'Directional'])
mask2 = main_add.LateralLength_pewh >= main_add.HorizontalDisplacement_hvs
main_add.loc[mask1 & mask2, 'LateralLength_pewh'] = main_add.loc[mask1 & mask2, 'HorizontalDisplacement_hvs']

main_add.loc[mask1, 'BuildRadius_pewh'] = main_add.loc[mask1, 'HorizontalDisplacement_hvs'] - main_add.loc[mask1, 'LateralLength_pewh']
main_add.loc[mask1, 'KOP_pewh'] = main_add.loc[mask1, 'TotalDepth'] - main_add.loc[mask1, 'LateralLength_pewh'] - main_add.loc[mask1, 'BuildRadius_pewh']*np.pi/2
main_add.loc[mask1, 'TVD_calc'] = main_add.loc[mask1, 'KOP_pewh'] + main_add.loc[mask1, 'BuildRadius_pewh']

main_add.loc[mask1, 'BuildRadiusRatio_pewh'] = main_add.loc[mask1, 'BuildRadius_pewh']/(main_add.loc[mask1, 'TotalDepth']-main_add.loc[mask1, 'LateralLength_pewh'])
main_add.loc[mask1, 'InclinationLocal_pewh'] = np.arcsin(main_add.loc[mask1, 'BuildRadiusRatio_pewh'])

In [43]:
# TVD_calc for vertical wells
mask = (main_add.WellProfile=='Vertical')
main_add.loc[mask, 'LateralLength_pewh'] = 0
main_add.loc[mask, 'BuildRadius_pewh'] = 0
main_add.loc[mask, 'KOP_pewh'] = main_add.loc[mask, 'TotalDepth']
main_add.loc[mask, 'TVD_calc'] = main_add.loc[mask, 'TotalDepth']

main_add.loc[mask, 'BuildRadiusRatio_pewh'] = 0
main_add.loc[mask, 'InclinationLocal_pewh'] = 0

In [48]:
# Additional ideas
main_add['HorizontalRatio_wh'] = main_add.HorizontalDisplacement_hvs/main_add.TotalDepth
main_add['LateralRatio_pewh'] = main_add.LateralLength_pewh/main_add.TotalDepth

main_add['InclinationTotal_wh'] = np.arcsin(main_add.HorizontalRatio_wh)
main_add['BuildRateTotal_pewh'] = main_add.InclinationTotal_wh/main_add.TotalDepth
main_add['BuildRateLocal_pewh'] = main_add.InclinationLocal_pewh/main_add.TotalDepth

main_add['KBHeight_wh'] = main_add.KBElevation - main_add.GroundElevation
main_add['TVDSS_calc'] = main_add.TVD_calc - main_add.KBElevation

##### TVDSS_pool

In [49]:
# TVDSS
main_add['TVDSS'] = main_add.TVD - main_add.KBElevation
main_add['FFP'] = main_add.Formation + main_add.Field + main_add.Pool

In [50]:
mask = (main_add.Formation!='Viking')
unspecified = main_add[(main_add.FFP.str.contains('Unspecified'))&mask].FFP.values
undefined = main_add[(main_add.FFP.str.contains('Undefined'))&mask].FFP.values
tempcomm = main_add[main_add.FFP.str.contains('Temporary Commingled')].FFP.values
misc = main_add[main_add.FFP.str.contains('Miscellaneous')].FFP.values
montney = ['MontneyAnte Creek NorthTriassic C', 'MontneyElmworthMontney JJJ', 'MontneyElmworthMontney U', 'MontneyPlacidMontney F', 
           'MontneyPouce Coupe SouthMontney YYY', 'MontneyPouce Coupe SouthCommingled Pool 012', 'MontneyWaskahiganMontney F']
unknown = main_add[main_add.FFP.isin(set(main_add[main_add.TVD.isnull()].FFP) - set(main_add[main_add.TVD.isnull()==False].FFP))].FFP.values

In [53]:
# Pool distance matrix (TVDSS)
# Use FFP TVDSS as Pool distance
df = main_add[['Formation', 'Field', 'Pool', 'TVDSS']].copy(deep=True)
ffp = df.groupby(['Formation', 'Field', 'Pool'])['TVDSS'].mean().to_frame(name='TVDSSmean').reset_index()
ffp['TVDSSmin'] = df.groupby(['Formation', 'Field', 'Pool'])['TVDSS'].min().values
ffp['TVDSSmax'] = df.groupby(['Formation', 'Field', 'Pool'])['TVDSS'].max().values
ffp['FFP'] = ffp.Formation + ffp.Field + ffp.Pool
# ffp.to_excel('ffp.xlsx', index=False)
ffp

,Formation,Field,Pool,TVDSSmean,TVDSSmin,TVDSSmax,FFP
0,Cardium,Anderson,Cardium B,1370.370000,1370.37,1370.37,CardiumAndersonCardium B
8,Cardium,Brazeau River,Cardium AAA,995.968333,981.46,1004.90,CardiumBrazeau RiverCardium AAA
9,Cardium,Brazeau River,Cardium BBB,1560.320000,1560.32,1560.32,CardiumBrazeau RiverCardium BBB
10,Cardium,Brazeau River,Cardium ZZ,1197.760000,1197.76,1197.76,CardiumBrazeau RiverCardium ZZ
13,Cardium,Caroline,Commingled Pool 002,975.741429,961.23,1007.40,CardiumCarolineCommingled Pool 002
...,...,...,...,...,...,...,...
840,Viking,Three Hills Creek,Viking BB,610.580000,610.58,610.58,VikingThree Hills CreekViking BB
841,Viking,Totnes,Totnes Viking Gas,23.105714,13.50,29.10,VikingTotnesTotnes Viking Gas
842,Viking,Verendrye,Verendrye Viking,24.012500,23.05,24.86,VikingVerendryeVerendrye Viking
844,Viking,Whiteside Viking,Whiteside Viking,22.438670,1.41,50.74,VikingWhiteside VikingWhiteside Viking


In [54]:
# Some Pool can be approx. by neighbor Pool
var = ['TVDSSmean', 'TVDSSmin', 'TVDSSmax']

mask = (ffp.Field=='Cygnet')
ffp.loc[mask&(ffp.Pool=='Duvernay M'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay N'), var].values

mask = (ffp.Field=='Elnora')
ffp.loc[mask&(ffp.Pool=='Duvernay C'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay D'), var].values

mask = (ffp.Field=='Gilby')
ffp.loc[mask&(ffp.Pool=='Duvernay D'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay C'), var].values

mask = (ffp.Field=='Grizzly')
ffp.loc[mask&(ffp.Pool=='Duvernay B'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Grizzly')&(main_add.Pool=='Duvernay B'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Duvernay B'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Grizzly')&(main_add.Pool=='Duvernay B'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Duvernay B'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Grizzly')&(main_add.Pool=='Duvernay B'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Huxley')
ffp.loc[mask&(ffp.Pool=='Duvernay F'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay E'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay J'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay I'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay M'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay I'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay N'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay I'), var].values

mask = (ffp.Field=='Joffre')
ffp.loc[mask&(ffp.Pool=='Duvernay DD'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay CC'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay X'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay V'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay Y'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay Z'), var].values

mask = (ffp.Field=='Kaybob')
ffp.loc[mask&(ffp.Pool=='Duvernay UUU'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay VVV'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay ZZ'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay YY'), var].values

mask = (ffp.Field=='Kaybob South')
ffp.loc[mask&(ffp.Pool=='Duvernay MMM'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay LLL'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay ZZZ'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay A2A'), var].values

mask = (ffp.Field=='Pembina')
ffp.loc[mask&(ffp.Pool=='Duvernay I'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Pembina')&(main_add.Pool=='Duvernay I'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Duvernay I'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Pembina')&(main_add.Pool=='Duvernay I'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Duvernay I'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Pembina')&(main_add.Pool=='Duvernay I'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Simonette')
ffp.loc[mask&(ffp.Pool=='Duvernay D'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay C'), var].values

mask = (ffp.Field=='Sylvan Lake')
ffp.loc[mask&(ffp.Pool=='Duvernay C'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay B'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay D'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay B'), var].values
ffp.loc[mask&(ffp.Pool=='Duvernay E'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay B'), var].values

mask = (ffp.Field=='Tony Creek North')
ffp.loc[mask&(ffp.Pool=='Duvernay O'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay N'), var].values

mask = (ffp.Field=='Twining')
ffp.loc[mask&(ffp.Pool=='Duvernay C'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay B'), var].values

mask = (ffp.Field=='Waskahigan')
ffp.loc[mask&(ffp.Pool=='Duvernay Y'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay X'), var].values

mask = (ffp.Field=='Ante Creek North')
ffp.loc[mask&(ffp.Pool=='Montney AA'), var] = ffp.loc[mask&(ffp.Pool=='Montney Q'), var].values

mask = (ffp.Field=='Elmworth')
ffp.loc[mask&(ffp.Pool=='Montney C4C'), var] = ffp.loc[mask&(ffp.Pool=='Montney D4D'), var].values
ffp.loc[mask&(ffp.Pool=='Montney H5H'), var] = ffp.loc[mask&(ffp.Pool=='Montney G5G'), var].values
ffp.loc[mask&(ffp.Pool=='Montney N5N'), var] = ffp.loc[mask&(ffp.Pool=='Montney N4N'), var].values
ffp.loc[mask&(ffp.Pool=='Montney Q5Q'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Elmworth')&(main_add.Pool=='Montney Q5Q'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney Q5Q'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Elmworth')&(main_add.Pool=='Montney Q5Q'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney Q5Q'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Elmworth')&(main_add.Pool=='Montney Q5Q'), 'TVDSS_calc'].max()
ffp.loc[mask&(ffp.Pool=='Montney R5R'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Elmworth')&(main_add.Pool=='Montney R5R'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney R5R'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Elmworth')&(main_add.Pool=='Montney R5R'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney R5R'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Elmworth')&(main_add.Pool=='Montney R5R'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Karr')
ffp.loc[mask&(ffp.Pool=='Montney A2A'), var] = ffp.loc[mask&(ffp.Pool=='Montney YYY'), var].values
ffp.loc[mask&(ffp.Pool=='Montney K2K'), var] = ffp.loc[mask&(ffp.Pool=='Montney H2H'), var].values
ffp.loc[mask&(ffp.Pool=='Montney ZZZ'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Karr')&(main_add.Pool=='Montney ZZZ'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney ZZZ'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Karr')&(main_add.Pool=='Montney ZZZ'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney ZZZ'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Karr')&(main_add.Pool=='Montney ZZZ'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Latornell')
ffp.loc[mask&(ffp.Pool=='Montney-Lower Doig Undefined'), var] = ffp.loc[(ffp.Field=='Pouce Coupe South')& (ffp.Pool=='Doig Undefined'), var].values + 250

mask = (ffp.Field=='Leland')
ffp.loc[mask&(ffp.Pool=='Montney K'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Leland')&(main_add.Pool=='Montney K'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney K'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Leland')&(main_add.Pool=='Montney K'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney K'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Leland')&(main_add.Pool=='Montney K'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Pouce Coupe')
ffp.loc[mask&(ffp.Pool=='Montney CCC'), var] = ffp.loc[mask&(ffp.Pool=='Montney BBB'), var].values 
ffp.loc[mask&(ffp.Pool=='Montney EEE'), var] = ffp.loc[mask&(ffp.Pool=='Montney DDD'), var].values
ffp.loc[mask&(ffp.Pool=='Montney FFF'), var] = ffp.loc[mask&(ffp.Pool=='Montney HHH'), var].values
ffp.loc[mask&(ffp.Pool=='Montney GGG'), var] = ffp.loc[mask&(ffp.Pool=='Montney HHH'), var].values
ffp.loc[mask&(ffp.Pool=='Montney SS'), var] = ffp.loc[mask&(ffp.Pool=='Montney TT'), var].values

mask = (ffp.Field=='Pouce Coupe South')
ffp.loc[mask&(ffp.Pool=='Montney D4D'), var] = ffp.loc[mask&(ffp.Pool=='Montney G4G'), var].values
ffp.loc[mask&(ffp.Pool=='Montney D9D'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Pouce Coupe South')&(main_add.Pool=='Montney D9D'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney D9D'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Pouce Coupe South')&(main_add.Pool=='Montney D9D'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney D9D'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Pouce Coupe South')&(main_add.Pool=='Montney D9D'), 'TVDSS_calc'].max()
ffp.loc[mask&(ffp.Pool=='Montney E7E'), var] = ffp.loc[mask&(ffp.Pool=='Montney G7G'), var].values
ffp.loc[mask&(ffp.Pool=='Montney E9E'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Pouce Coupe South')&(main_add.Pool=='Montney E9E'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney E9E'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Pouce Coupe South')&(main_add.Pool=='Montney E9E'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney E9E'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Pouce Coupe South')&(main_add.Pool=='Montney E9E'), 'TVDSS_calc'].max()
ffp.loc[mask&(ffp.Pool=='Montney G8G'), var] = ffp.loc[mask&(ffp.Pool=='Montney H8H'), var].values 
ffp.loc[mask&(ffp.Pool=='Montney I5I'), var] = ffp.loc[mask&(ffp.Pool=='Montney I8I'), var].values 
ffp.loc[mask&(ffp.Pool=='Montney J5J'), var] = ffp.loc[mask&(ffp.Pool=='Montney G5G'), var].values 
ffp.loc[mask&(ffp.Pool=='Montney M4M'), var] = ffp.loc[mask&(ffp.Pool=='Montney M7M'), var].values
ffp.loc[mask&(ffp.Pool=='Montney N4N'), var] = ffp.loc[mask&(ffp.Pool=='Montney J4J'), var].values
ffp.loc[mask&(ffp.Pool=='Montney V5V'), var] = ffp.loc[mask&(ffp.Pool=='Montney W5W'), var].values
ffp.loc[mask&(ffp.Pool=='Montney V7V'), var] = ffp.loc[mask&(ffp.Pool=='Montney W7W'), var].values
ffp.loc[mask&(ffp.Pool=='Montney W8W'), var] = ffp.loc[mask&(ffp.Pool=='Montney W7W'), var].values
ffp.loc[mask&(ffp.Pool=='Montney X7X'), var] = ffp.loc[mask&(ffp.Pool=='Montney Y6Y'), var].values
ffp.loc[mask&(ffp.Pool=='Montney X8X'), var] = ffp.loc[mask&(ffp.Pool=='Montney X6X'), var].values
ffp.loc[mask&(ffp.Pool=='Montney Z4Z'), var] = ffp.loc[mask&(ffp.Pool=='Montney Z6Z'), var].values

mask = (ffp.Field=='Progress')
ffp.loc[mask&(ffp.Pool=='Montney M'), var] = ffp.loc[mask&(ffp.Pool=='Montney N'), var].values
ffp.loc[mask&(ffp.Pool=='Montney S'), var] = ffp.loc[mask&(ffp.Pool=='Montney R'), var].values

mask = (ffp.Field=='Valhalla')
ffp.loc[mask&(ffp.Pool=='Montney D4D'), var] = ffp.loc[mask&(ffp.Pool=='Montney C4C'), var].values
ffp.loc[mask&(ffp.Pool=='Montney T3T'), var] = ffp.loc[mask&(ffp.Pool=='Montney V3V'), var].values
ffp.loc[mask&(ffp.Pool=='Montney U3U'), var] = ffp.loc[mask&(ffp.Pool=='Montney V3V'), var].values

mask = (ffp.Field=='Wapiti')
ffp.loc[mask&(ffp.Pool=='Montney DD'), var] = ffp.loc[mask&(ffp.Pool=='Montney EE'), var].values
ffp.loc[mask&(ffp.Pool=='Montney QQ'), var] = ffp.loc[mask&(ffp.Pool=='Montney OO'), var].values
ffp.loc[mask&(ffp.Pool=='Montney RR'), var] = ffp.loc[mask&(ffp.Pool=='Montney OO'), var].values
ffp.loc[mask&(ffp.Pool=='Montney VV'), var] = ffp.loc[mask&(ffp.Pool=='Montney TT'), var].values

mask = (ffp.Field=='Wembley')
ffp.loc[mask&(ffp.Pool=='Montney AAA'), var] = ffp.loc[mask&(ffp.Pool=='Montney XX'), var].values
ffp.loc[mask&(ffp.Pool=='Montney C2C'), var] = ffp.loc[mask&(ffp.Pool=='Montney D2D'), var].values
ffp.loc[mask&(ffp.Pool=='Montney NN'), var] = ffp.loc[mask&(ffp.Pool=='Montney MM'), var].values
ffp.loc[mask&(ffp.Pool=='Montney S2S'), var] = ffp.loc[mask&(ffp.Pool=='Montney T2T'), var].values
ffp.loc[mask&(ffp.Pool=='Montney YYY'), var] = ffp.loc[mask&(ffp.Pool=='Montney ZZZ'), var].values

mask = (ffp.Field=='Loverna')
ffp.loc[mask, 'TVDSSmean'] = main_add.loc[main_add.Pool.str.contains('Viking'), 'TVDSS'].mean()
ffp.loc[mask, 'TVDSSmin'] = main_add.loc[main_add.Pool.str.contains('Viking'), 'TVDSS'].min()
ffp.loc[mask, 'TVDSSmax'] = main_add.loc[main_add.Pool.str.contains('Viking'), 'TVDSS'].max()

mask = (ffp.Field=='Peco')
ffp.loc[mask&(ffp.Pool=='Viking K'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Peco')&(main_add.Pool=='Viking K'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Viking K'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Peco')&(main_add.Pool=='Viking K'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Viking K'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Peco')&(main_add.Pool=='Viking K'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Provost')
ffp.loc[mask&(ffp.Pool=='Viking I9I'), var] = ffp.loc[mask&(ffp.Pool=='Viking F9F'), var].values
ffp.loc[mask&(ffp.Pool=='Viking Q9Q'), var] = ffp.loc[mask&(ffp.Pool=='Viking O9O'), var].values

In [55]:
# Undefined, Unspecifed, Temporary Commingled

mask = (ffp.Field=='Chigwell')
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Chigwell')&(main_add.Pool=='Duvernay Undefined'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Chigwell')&(main_add.Pool=='Duvernay Undefined'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Chigwell')&(main_add.Pool=='Duvernay Undefined'), 'TVDSS_calc'].max()
ffp.loc[mask&(ffp.Pool=='Duvernay A'), var] = ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), var].values

mask = (ffp.Field=='Ferrybank')
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Ferrybank')&(main_add.Pool=='Duvernay Undefined'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Ferrybank')&(main_add.Pool=='Duvernay Undefined'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Ferrybank')&(main_add.Pool=='Duvernay Undefined'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Gilby')
ffp.loc[mask&(ffp.Pool=='Temporary Commingled'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Gilby')&(main_add.Pool=='Temporary Commingled'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Temporary Commingled'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Gilby')&(main_add.Pool=='Temporary Commingled'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Temporary Commingled'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Gilby')&(main_add.Pool=='Temporary Commingled'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Two Creek')
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmean'] = main_add.loc[main_add.Field=='Two Creek', 'TVDSS'].mean()
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmin'] = main_add.loc[main_add.Field=='Two Creek', 'TVDSS'].min()
ffp.loc[mask&(ffp.Pool=='Duvernay Undefined'), 'TVDSSmax'] = main_add.loc[main_add.Field=='Two Creek', 'TVDSS'].max()

mask = (ffp.Field=='Ante Creek North')
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Ante Creek North')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Ante Creek North')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Ante Creek North')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Fox Creek')
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Fox Creek')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Fox Creek')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Fox Creek')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].max()

mask = (ffp.Field=='Grande Prairie')
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmean'] = main_add.loc[(main_add.Field=='Grande Prairie')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].mean()
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmin'] = main_add.loc[(main_add.Field=='Grande Prairie')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].min()
ffp.loc[mask&(ffp.Pool=='Montney Undefined'), 'TVDSSmax'] = main_add.loc[(main_add.Field=='Grande Prairie')&(main_add.Pool=='Montney Undefined'), 'TVDSS_calc'].max()

In [60]:
ffp.dropna(inplace=True)

In [62]:
df = main_add[['Formation', 'Field', 'Pool']].copy(deep=True)
df = df.merge(ffp[['Formation', 'Field', 'Pool', 'TVDSSmean']], on=['Formation', 'Field', 'Pool'], how='left')
df = df.drop(['Formation', 'Field', 'Pool'], axis=1)
pool_matrix = cdist(df, df, metric='euclidean')
print(np.nanmean(pool_matrix))
print(np.nanmin(pool_matrix))
print(np.nanmax(pool_matrix))

735.5198082218074
0.0
2935.006666666666


In [52]:
# Weak Pool and weak wells
WPS = list(set().union(*[unspecified, undefined, tempcomm, misc, montney, unknown]))
WPS = list(set(WPS) - set(ffp.FFP.values))
WWS = main_add[main_add.FFP.isin(WPS)].EPAssetsId.values
# Strong Pool and strong wells
SPS = list(set(main_add.FFP) - set(WPS))
SWS = main_add[main_add.FFP.isin(SPS)].EPAssetsId.values

print(len(WPS))
print(len(WWS))
print(len(SPS))
print(len(SWS))

174
1850
672
6839


In [66]:
# Mean, min, max TVDSS of FFP
main_add = main_add.merge(ffp[['Formation', 'Field', 'Pool', 'TVDSSmean', 'TVDSSmin', 'TVDSSmax']],
                          on=['Formation', 'Field', 'Pool'], how='left')
main_add.rename(columns={'TVDSSmean': 'TVDSS_poolmean', 'TVDSSmin': 'TVDSS_poolmin', 'TVDSSmax': 'TVDSS_poolmax'},
                inplace=True)

#### Neighbors Features

In [67]:
# For neighbor features of TVDSS, wellprofile is NOT needed
totaldistance_matrix2 = distance_matrix + pool_matrix

In [68]:
# strong wells x train (train is NOT subset of strong due to unspecified, undefined etc.)
totaldistance2 = pd.DataFrame(totaldistance_matrix2, index=main_add.EPAssetsId, columns=main_add.EPAssetsId)
strongwells = main_add[main_add.FFP.isin(SPS)].EPAssetsId.values
trainwells = main_add.loc[main_add.TVD.isnull()==False, 'EPAssetsId'].values
rowmask = totaldistance2.index[totaldistance2.index.isin(strongwells)]
colmask = totaldistance2.columns[totaldistance2.columns.isin(trainwells)]
totaldistance2 = totaldistance2.loc[rowmask, colmask]
totaldistance2.shape

(6839, 6726)

In [69]:
totaldistance2 = totaldistance2.values
n_neighbors = 50
neighbors2 = np.empty((totaldistance2.shape[0], n_neighbors))
for i in range(0, totaldistance2.shape[0]): 
    neighbors2[i, :] = list(trainwells[np.argsort(totaldistance2[i, :])[:n_neighbors]])
neighbors2.shape

(6839, 50)

In [70]:
# ONLY include STRONG wells
main_add = main_add[main_add.EPAssetsId.isin(strongwells)].reset_index(drop=True)

In [71]:
# TVDSS from neighbor wells (take some time)
for n_neighbors in range(1, 6):
    for index in main_add.index:        
        if pd.isnull(main_add.loc[main_add.index == index, 'TVD']).values[0]: 
            # for test well, count from 1st nearest
            mask = main_add.EPAssetsId.isin(neighbors2[index, 0:n_neighbors])
        else:
            # for train well, count from 2nd nearest as 1st nearest is itself
            mask = main_add.EPAssetsId.isin(neighbors2[index, 1:n_neighbors+1])
        main_add.loc[main_add.index == index, 'TVDSS_mean'+str(n_neighbors).zfill(2)] = main_add.loc[mask, 'TVDSS'].mean()
        main_add.loc[main_add.index == index, 'TVDSS_min'+str(n_neighbors).zfill(2)] = main_add.loc[mask, 'TVDSS'].min()
        main_add.loc[main_add.index == index, 'TVDSS_max'+str(n_neighbors).zfill(2)] = main_add.loc[mask, 'TVDSS'].max()
    print(main_add['TVDSS_mean'+str(n_neighbors).zfill(2)].isnull().sum())
    print(main_add['TVDSS_min'+str(n_neighbors).zfill(2)].isnull().sum())
    print(main_add['TVDSS_max'+str(n_neighbors).zfill(2)].isnull().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [72]:
# TVDSS_calc from neighbor wells (take some time)
for n_neighbors in range(1, 6):
    for index in main_add.index:        
        if pd.isnull(main_add.loc[main_add.index == index, 'TVD']).values[0]: 
            # for test well, count from 1st nearest
            mask = main_add.EPAssetsId.isin(neighbors2[index, 0:n_neighbors])
        else:
            # for train well, count from 2nd nearest as 1st nearest is itself
            mask = main_add.EPAssetsId.isin(neighbors2[index, 1:n_neighbors+1])        
        main_add.loc[main_add.index == index, 'TVDSS_calcmean'+str(n_neighbors).zfill(2)] = main_add.loc[mask, 'TVDSS_calc'].mean()
        main_add.loc[main_add.index == index, 'TVDSS_calcmin'+str(n_neighbors).zfill(2)] = main_add.loc[mask, 'TVDSS_calc'].min()
        main_add.loc[main_add.index == index, 'TVDSS_calcmax'+str(n_neighbors).zfill(2)] = main_add.loc[mask, 'TVDSS_calc'].max()
    print(main_add['TVDSS_calcmean'+str(n_neighbors).zfill(2)].isnull().sum())
    print(main_add['TVDSS_calcmin'+str(n_neighbors).zfill(2)].isnull().sum())
    print(main_add['TVDSS_calcmax'+str(n_neighbors).zfill(2)].isnull().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


#### TVDSS_rgrs

In [ ]:
# TVDSS regression from neighbor TVDSS
from sklearn.linear_model import LinearRegression

mask = (main_add.TVDSS.isnull()==False) & (main_add.Formation.isin(['Duvernay', 'Montney', 'Viking']))
lr = LinearRegression()
x = np.column_stack((main_add[mask].TVDSS_mean01.values.reshape(-1,1), 
                     main_add[mask].TVDSS_mean02.values.reshape(-1,1), 
                     main_add[mask].TVDSS_mean03.values.reshape(-1,1), 
                     main_add[mask].TVDSS_mean04.values.reshape(-1,1), 
                     main_add[mask].TVDSS_mean05.values.reshape(-1,1)))
lr.fit(x, main_add[mask].TVDSS.values)

In [84]:
main_add['TVDSS_rgrs'] = lr.predict(np.column_stack((main_add.TVDSS_mean01.values.reshape(-1,1), 
                                                     main_add.TVDSS_mean02.values.reshape(-1,1), 
                                                     main_add.TVDSS_mean03.values.reshape(-1,1), 
                                                     main_add.TVDSS_mean04.values.reshape(-1,1), 
                                                     main_add.TVDSS_mean05.values.reshape(-1,1))))

#### DateTime and Categorical

In [86]:
# Convert datetime to days
timeref = pd.to_datetime('01012000', format='%d%m%Y')
for var in main_add.columns[main_add.dtypes=='datetime64[ns]']: 
    main_add[var+'_day'] = (main_add[var] - timeref).dt.days
    main_add = main_add.drop(var, axis=1)

In [87]:
# Rig Working Time
main_add['RigWorkTime_wh'] = main_add.FinalDrillDate_day - main_add.SpudDate_day
# Rig Idle Time
main_add['RigIdleTime_wh'] = main_add.RigReleaseDate_day - main_add.FinalDrillDate_day
# Well Age Paper
main_add['WellAgePaper_wh'] = main_add.StatusDate_day - main_add.LicenceDate_day
# Well Age Real
main_add['WellAgeReal_wh'] = main_add.StatusDate_day - main_add.SpudDate_day
# Well Condidential Time Paper
main_add['ConfidentialTimePaper_wh'] = main_add.ConfidentialReleaseDate_day - main_add.LicenceDate_day
# Well Condidential Time Real
main_add['ConfidentialTimeReal_wh'] = main_add.ConfidentialReleaseDate_day - main_add.SpudDate_day

In [88]:
# Well Idle Times
main_add['DrillCompleteIdleTime_pewh'] = main_add.CompletionStartDate_perf_day - main_add.FinalDrillDate_day
main_add['CompleteProdIdleTime_pepr'] = main_add.StartDate_prod_day - main_add.CompletionEndDate_perf_day
main_add['TotalIdleTime_pewh'] = main_add.DrillCompleteIdleTime_pewh + main_add.CompleteProdIdleTime_pepr

#### Export

In [89]:
# Check NaN
datatype = pd.DataFrame(main_add.dtypes, columns=['pdtype'])
datatype['nunique'] = main_add.nunique()
datatype['nnan'] = main_add.isnull().sum()
datatype

,pdtype,nunique,nnan
EPAssetsId,int64,6839,0
Province,object,2,0
LicenceNumber,object,6835,0
UWI,object,6839,0
CurrentOperator,object,96,0
CurrentOperatorParent,object,95,0
CurrentOperatorParentName,object,95,0
Licensee,object,92,0
LicenseeParentCompany,object,92,0
LicenseeParentCompanyName,object,92,0


In [90]:
main_add = main_add.drop(['UnitFlag', 'UnitName', 'UnitID', 'Municipality', 'StatusSource', 'RegulatoryAgency'], axis=1)

In [91]:
main_add.shape

(6839, 144)

In [92]:
main_add.to_csv('Xy_Neigh10Add10Strong_rgrs4_tvdss.csv', index=False)